Cylleneus + NLP
===============

Probably most often, the Cylleneus search engine will be used through one of its more user-friendly interfaces. However, it is also possible to use the engine as an API and to perform queries programmatically. In this way, the engine can in fact be used to build NLP applications. One very simple and straightforward use of Cylleneus' lemma-based query functionality would be to try to find 'intertexts' -- passages of text that are lexically similar to, but not morphologically identical to, some source text. Let's try it out.

First, set up the search environment.

In [1]:
# We need tell Cylleneus what corpus we want to search, and then instantiate a Searcher object to execute specific queries.
from corpus import Corpus
from search import Searcher

Let's use the Perseus Digital Library minicorpus that comes pre-installed with the Cylleneus repository; it includes the major works of Vergil.

In [2]:
corpus = Corpus('perseus')

Because we want to abstract away morphological details of our source text, we are also going to need to tokenize and lemmatize this text. In this case, for simplicity's sake, we will just be searching for a single phrase, which we can input manually. Since the text isn't coming from a structured corpus, we can use the built-in plaintext tokenizer and lemmatizer.

In [3]:
# The plaintext tokenizer should be suitable for just about any 'raw' Latin text.
from engine.analysis.tokenizers import CachedPlainTextTokenizer

# The lemma filter takes a sequence of tokens (word-forms) and uses the Latin WordNet for lemmatization and morphological analysis.
from engine.analysis.filters import CachedLemmaFilter

tokenizer = CachedPlainTextTokenizer()
lemmatizer = CachedLemmaFilter()

Now let's run our source text through our lemmatization pipeline. In this fabricated example, we are going to search for texts similar to the phrase of Lucretius: *gelidamque pruinam* (Lucr. *RN.* 2.431).

In [58]:
# For efficiency the tokenizer reuses a single Token object, so each token needs to be copied to be preserved
from copy import copy

text = 'gelidamque pruinam'
tokens = [copy(token) for token in tokenizer(text, mode='index')]

lemmas = []
for token in tokens:
    lemmas.append(list(set([lemma.text for lemma in lemmatizer([token,], mode='query')])))

At this point we need to construct a well-formed lemma-based query for Cylleneus to execute. In the most basic kind of query, we would simply combine the lemmatized tokens together as a sequence.

NB. The lemmatizer tries to be inclusive as possible, so a form like *fatis* will generate multiple lemmas for possible matching: *fatum* as well as *fatis* and *fatus*. This is why, if we were to inspect the `lemmas` object, we would find that each word of the original text resolves to a list of lemmas.

In [59]:
from pprint import pprint
pprint(lemmas)

[['gelida=n-s---fn1-', 'gelidus=aps---mn1-'],
 ['que=rp--------'],
 ['pruina=n-s---fn1-']]


In [60]:
# Construct sequential lemma-based query
subqueries = []
for i, lemma in enumerate(lemmas):
    if len(lemma) == 0:  # no lemma found, use the original form
        subqueries.append(tokens[i].text)
    elif len(lemma) == 1:
        subqueries.append(f"<{lemma[0]}>")
    else:
        subqueries.append(f'''({' OR '.join([f"<{alt}>" for alt in lemma])})''')

# Join all subqueries into a single adjacency query
adjacent_lemmas = f'''"{' '.join(subqueries)}"'''

To be more inclusive, we could do away entire with the sequential requirement and try instead using a proximity query. In this case, any text will match provided only that it contains the matching query terms, irrespective of their ordering

In [61]:
proximal_lemmas = f'''{' '.join(subqueries)}'''

In [62]:
# Execute the query against the given corpus. `minscore` guarantees that only results
# matching all three terms will be returned
searcher = Searcher(corpus)
search = searcher.search(proximal_lemmas)  

# Display the query if any matches
if search.count != (0, 0):  # matches, docs
    pprint(list(search.highlights))

['\n',
 'VIRGIL, GEORGICS\n',
 'poem: 2, line: 367-poem: 2, line: 368\n',
 '    intereunt pecudes, stant circumfusa pruinis\n',
 '\n',
 'poem: 1, line: 487\n',
 '    Taygeta! O, qui me gelidis convallibus Haemi\n',
 '\n',
 'poem: 3, line: 508\n',
 '    flesse sibi et gelidis haec evolvisse sub antris\n',
 '\n',
 'poem: 0, line: 286\n',
 '    Multa adeo gelida melius se nocte dedere,\n',
 '\n',
 'poem: 1, line: 262\n',
 '    arva solo: id venti curant gelidaeque pruinae\n',
 '\n']
